In [1]:
import sys
sys.path.append(str('/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/'))

import pandas as pd
from configparser import ConfigParser
from new_repo.MyConfigParser import MyConfigParser
from new_repo.LoadData import DataLoader
from new_repo.BuildModel import GenerateModelFile, CreateModel
from new_repo.Optimization import SavantPSO


In [2]:

config_in_path = "/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/Account type/ca/24 Sep Update/config ups CA 24 Sep.ini"
config_file = MyConfigParser(config_in_path)
config_dict = config_file.GetDict()

region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]

base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = base_path + config_dict["FILEPATH"]["constraint_file_name"]
breakdown_ls = [config_dict['CLIENTINFO'][i] for i in [f'breakdown{i}' for i in range(1,5)] if config_dict['CLIENTINFO'][i] != None]

small_market_str = ''
pre_forecast_period_budget = float(config_dict['CLIENTINFO']['pre_forecast_period_budget'])
client_budget = float(config_dict['CLIENTINFO']['client_budget'])

save_date = config_dict['CLIENTINFO']['save_date']

In [3]:
# Get forecasters
bdDf = pd.read_csv(cuts_df_in_path)#.drop('Unnamed: 0', axis = 1)
cuts_df = bdDf.reset_index(drop = True)
cuts_df['forecaster_name_list'] =  cuts_df['market'] + ' ' + cuts_df['account_type'] + ' ' + cuts_df['funnel'] + '.pkl' #' 05.19.2021 Final.pkl'
cuts_df['periods'] = periods
forecasters_array = cuts_df.forecaster_name_list

# Get analytical file
analytical_file = pd.read_csv(analytical_file_path).drop('Unnamed: 0', axis = 1)
analytical_file['ds'] = pd.to_datetime(analytical_file['ds'])
analytical_file_check_version = analytical_file.copy()

# Get constrain file
constraint_file = pd.read_csv(constraint_file_name)

In [4]:
today = '01.22.2024' #datetime.date.today().strftime("%m.%d.%Y")# 
fileName = base_path + f'Pos Result {today}.csv' #today
pos_df = pd.read_csv(fileName).drop('Unnamed: 0', axis = 1)

In [5]:
resultPSO = new_repo.Optimization.SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights = pos_df.pos.to_numpy()

daily_simulated_forecast = resultPSO.ResultGenerator(inputWeights)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  687361.0 difference between planned and forecast X:  -0.0
yhat: 255733.0
analytical_spend_latest_period: 636401.9999999998
client_budget - analytical_spend_latest_period: 0


In [6]:
# %load_ext autoreload
# %autoreload 2
resultPSO = new_repo.Optimization.SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights_base = np.ones(constraint_file.shape[0])

daily_simulated_forecast_base = resultPSO.ResultGenerator(inputWeights_base)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  687361.0 difference between planned and forecast X:  0.0
yhat: 248409.0
analytical_spend_latest_period: 636401.9999999999
client_budget - analytical_spend_latest_period: 0


In [7]:
forecasted_daily_df = daily_simulated_forecast[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
forecasted_daily_df['month'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
forecasted_daily_df['year'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%Y').astype(str)

forecasted_daily_df = forecasted_daily_df.rename(columns = { 'yhat': 'OptimalConversionQuantity', 'X': 'OptimalSpendValue'})
forecasted_daily_df['OptimalSpendValue'] = forecasted_daily_df['OptimalSpendValue'].round(2)

forecasted_daily_df['variable_splits'] = forecasted_daily_df['market'] + '_' + forecasted_daily_df['account_type']+'_'+forecasted_daily_df['funnel']+'_'+forecasted_daily_df['month']+'_'+forecasted_daily_df['year']
print(forecasted_daily_df.OptimalSpendValue.sum(), forecasted_daily_df.OptimalConversionQuantity.sum())
forecasted_daily_df

687360.6399999999 255733.0


,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits
0,Canada,-,Brand,2024-01-01,461.0,1182.27,Jan,2024,Canada_-_Brand_Jan_2024
1,Canada,-,Brand,2024-01-02,1006.0,2310.57,Jan,2024,Canada_-_Brand_Jan_2024
2,Canada,-,Brand,2024-01-03,772.0,1786.98,Jan,2024,Canada_-_Brand_Jan_2024
3,Canada,-,Brand,2024-01-04,629.0,1460.16,Jan,2024,Canada_-_Brand_Jan_2024
4,Canada,-,Brand,2024-01-05,631.0,1485.99,Jan,2024,Canada_-_Brand_Jan_2024
...,...,...,...,...,...,...,...,...,...
727,Canada,-,Non_Brand,2024-12-27,133.0,821.96,Dec,2024,Canada_-_Non_Brand_Dec_2024
728,Canada,-,Non_Brand,2024-12-28,93.0,551.92,Dec,2024,Canada_-_Non_Brand_Dec_2024
729,Canada,-,Non_Brand,2024-12-29,89.0,509.13,Dec,2024,Canada_-_Non_Brand_Dec_2024
730,Canada,-,Non_Brand,2024-12-30,115.0,607.49,Dec,2024,Canada_-_Non_Brand_Dec_2024


In [8]:
base_daily_df = daily_simulated_forecast_base[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
base_daily_df['month'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
base_daily_df['year'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%Y').astype(str)
base_daily_df['X'] = base_daily_df['X'].round(2)
base_daily_df = base_daily_df.rename(columns = { 'X': 'BaseSpend', 'yhat': 'BaseConversionQuantity'})

print(base_daily_df.BaseSpend.sum(), base_daily_df.BaseConversionQuantity.sum())
base_daily_df

687360.52 248409.0


,market,account_type,funnel,ds,BaseConversionQuantity,BaseSpend,month,year
0,Canada,-,Brand,2024-01-01,418.0,1091.54,Jan,2024
1,Canada,-,Brand,2024-01-02,922.0,2133.24,Jan,2024
2,Canada,-,Brand,2024-01-03,705.0,1649.84,Jan,2024
3,Canada,-,Brand,2024-01-04,573.0,1348.10,Jan,2024
4,Canada,-,Brand,2024-01-05,575.0,1371.95,Jan,2024
...,...,...,...,...,...,...,...,...
727,Canada,-,Non_Brand,2024-12-27,147.0,1009.19,Dec,2024
728,Canada,-,Non_Brand,2024-12-28,103.0,677.63,Dec,2024
729,Canada,-,Non_Brand,2024-12-29,98.0,625.10,Dec,2024
730,Canada,-,Non_Brand,2024-12-30,126.0,745.86,Dec,2024


In [9]:
forecast_n_base_df = pd.merge(forecasted_daily_df, base_daily_df, how = 'left', on = ['market', 'account_type', 'funnel', 'ds', 'month', 'year'])
forecast_n_base_df['SavedDate'] = datetime.datetime.strptime(save_date, '_%m_%d_%y').strftime("%Y-%m-%d")

forecast_n_base_df.tail()

,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits,BaseConversionQuantity,BaseSpend,SavedDate
727,Canada,-,Non_Brand,2024-12-27,133.0,821.96,Dec,2024,Canada_-_Non_Brand_Dec_2024,147.0,1009.19,2024-02-05
728,Canada,-,Non_Brand,2024-12-28,93.0,551.92,Dec,2024,Canada_-_Non_Brand_Dec_2024,103.0,677.63,2024-02-05
729,Canada,-,Non_Brand,2024-12-29,89.0,509.13,Dec,2024,Canada_-_Non_Brand_Dec_2024,98.0,625.10,2024-02-05
730,Canada,-,Non_Brand,2024-12-30,115.0,607.49,Dec,2024,Canada_-_Non_Brand_Dec_2024,126.0,745.86,2024-02-05
731,Canada,-,Non_Brand,2024-12-31,165.0,1240.07,Dec,2024,Canada_-_Non_Brand_Dec_2024,186.0,1522.53,2024-02-05


# Large Market

In [10]:
lm_df = forecast_n_base_df[forecast_n_base_df['market'] != 'SM'].copy()
lm_split_df = pd.merge(lm_df, analytical_file[['market', 'funnel', 'ds', 'generic_cost_split', 'competitor_cost_split']], how = 'left', on = ['market', 'funnel', 'ds'])

# helper_df is used to get the investment split for each funnel
lm_generic_helper_df = lm_split_df.loc[lm_split_df['funnel'] == 'Non_Brand', ].copy()
lm_generic_helper_df.loc[:,'funnel'] = 'Generic'
lm_generic_helper_df.loc[:,'OptimalSpendValue'] = lm_generic_helper_df['OptimalSpendValue'] * lm_generic_helper_df['generic_cost_split']
lm_generic_helper_df.loc[:,'OptimalConversionQuantity'] = lm_generic_helper_df['OptimalConversionQuantity'] * lm_generic_helper_df['generic_cost_split']#round(lm_generic_helper_df['OptimalConversionQuantity'] * lm_generic_helper_df['generic_cost_split'], 0)
lm_generic_helper_df.loc[:,'BaseSpend'] = lm_generic_helper_df['BaseSpend'] * lm_generic_helper_df['generic_cost_split']
lm_generic_helper_df.loc[:,'BaseConversionQuantity'] = lm_generic_helper_df['BaseConversionQuantity'] * lm_generic_helper_df['generic_cost_split'] #round(lm_generic_helper_df['BaseConversionQuantity'] * lm_generic_helper_df['generic_cost_split'], 0)

lm_competitor_helper_df = lm_split_df.loc[lm_split_df['funnel'] == 'Non_Brand', ].copy()
lm_competitor_helper_df.loc[:,'funnel'] = 'Competitor'
lm_competitor_helper_df.loc[:,'OptimalSpendValue'] = lm_competitor_helper_df['OptimalSpendValue'] * lm_competitor_helper_df['competitor_cost_split']
lm_competitor_helper_df.loc[:,'OptimalConversionQuantity'] = lm_competitor_helper_df['OptimalConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'] #round(lm_competitor_helper_df['OptimalConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'], 0)
lm_competitor_helper_df.loc[:,'BaseSpend'] = lm_competitor_helper_df['BaseSpend'] * lm_competitor_helper_df['competitor_cost_split']
lm_competitor_helper_df.loc[:,'BaseConversionQuantity'] = lm_competitor_helper_df['BaseConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'] #round(lm_competitor_helper_df['BaseConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'], 0)

lm_funnel_level_daily_df = pd.concat([lm_split_df[(lm_split_df['funnel'] == 'Brand') | (lm_split_df['funnel'] == '-')], lm_generic_helper_df, lm_competitor_helper_df])
print(lm_funnel_level_daily_df.market.unique())
lm_funnel_level_daily_df

['Canada']


,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits,BaseConversionQuantity,BaseSpend,SavedDate,generic_cost_split,competitor_cost_split
0,Canada,-,Brand,2024-01-01,461.000000,1182.270000,Jan,2024,Canada_-_Brand_Jan_2024,418.000000,1091.540000,2024-02-05,0.500000,0.500000
1,Canada,-,Brand,2024-01-02,1006.000000,2310.570000,Jan,2024,Canada_-_Brand_Jan_2024,922.000000,2133.240000,2024-02-05,0.500000,0.500000
2,Canada,-,Brand,2024-01-03,772.000000,1786.980000,Jan,2024,Canada_-_Brand_Jan_2024,705.000000,1649.840000,2024-02-05,0.500000,0.500000
3,Canada,-,Brand,2024-01-04,629.000000,1460.160000,Jan,2024,Canada_-_Brand_Jan_2024,573.000000,1348.100000,2024-02-05,0.500000,0.500000
4,Canada,-,Brand,2024-01-05,631.000000,1485.990000,Jan,2024,Canada_-_Brand_Jan_2024,575.000000,1371.950000,2024-02-05,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Canada,-,Competitor,2024-12-27,18.926001,116.965535,Dec,2024,Canada_-_Non_Brand_Dec_2024,20.918212,143.608506,2024-02-05,0.857699,0.142301
728,Canada,-,Competitor,2024-12-28,8.439516,50.085352,Dec,2024,Canada_-_Non_Brand_Dec_2024,9.346991,61.493218,2024-02-05,0.909253,0.090747
729,Canada,-,Competitor,2024-12-29,6.197005,35.450348,Dec,2024,Canada_-_Non_Brand_Dec_2024,6.823668,43.525254,2024-02-05,0.930371,0.069629
730,Canada,-,Competitor,2024-12-30,9.658757,51.022594,Dec,2024,Canada_-_Non_Brand_Dec_2024,10.582638,62.644178,2024-02-05,0.916011,0.083989


In [11]:
lm_funnel_level_daily_df = lm_funnel_level_daily_df.rename(columns=lambda x: x[0:1].upper() + x[1:])
lm_funnel_level_daily_df = lm_funnel_level_daily_df.drop(['Variable_splits', 'Generic_cost_split', 'Competitor_cost_split'], axis = 1)
lm_funnel_level_daily_df

,Market,Account_type,Funnel,Ds,OptimalConversionQuantity,OptimalSpendValue,Month,Year,BaseConversionQuantity,BaseSpend,SavedDate
0,Canada,-,Brand,2024-01-01,461.000000,1182.270000,Jan,2024,418.000000,1091.540000,2024-02-05
1,Canada,-,Brand,2024-01-02,1006.000000,2310.570000,Jan,2024,922.000000,2133.240000,2024-02-05
2,Canada,-,Brand,2024-01-03,772.000000,1786.980000,Jan,2024,705.000000,1649.840000,2024-02-05
3,Canada,-,Brand,2024-01-04,629.000000,1460.160000,Jan,2024,573.000000,1348.100000,2024-02-05
4,Canada,-,Brand,2024-01-05,631.000000,1485.990000,Jan,2024,575.000000,1371.950000,2024-02-05
...,...,...,...,...,...,...,...,...,...,...,...
727,Canada,-,Competitor,2024-12-27,18.926001,116.965535,Dec,2024,20.918212,143.608506,2024-02-05
728,Canada,-,Competitor,2024-12-28,8.439516,50.085352,Dec,2024,9.346991,61.493218,2024-02-05
729,Canada,-,Competitor,2024-12-29,6.197005,35.450348,Dec,2024,6.823668,43.525254,2024-02-05
730,Canada,-,Competitor,2024-12-30,9.658757,51.022594,Dec,2024,10.582638,62.644178,2024-02-05


In [12]:
LM_SM_three_funnel_df = pd.concat([pd.DataFrame(), lm_funnel_level_daily_df]).reset_index(drop = True)
LM_SM_three_funnel_df.insert(3, 'WeekStart', [ i for i in LM_SM_three_funnel_df.Ds.apply(lambda x: x - pd.offsets.Week(weekday=4) )])
LM_SM_three_funnel_df.insert(0, 'Region', region)
LM_SM_three_funnel_df

,Region,Market,Account_type,Funnel,WeekStart,Ds,OptimalConversionQuantity,OptimalSpendValue,Month,Year,BaseConversionQuantity,BaseSpend,SavedDate
0,Canada,Canada,-,Brand,2023-12-29,2024-01-01,461.000000,1182.270000,Jan,2024,418.000000,1091.540000,2024-02-05
1,Canada,Canada,-,Brand,2023-12-29,2024-01-02,1006.000000,2310.570000,Jan,2024,922.000000,2133.240000,2024-02-05
2,Canada,Canada,-,Brand,2023-12-29,2024-01-03,772.000000,1786.980000,Jan,2024,705.000000,1649.840000,2024-02-05
3,Canada,Canada,-,Brand,2023-12-29,2024-01-04,629.000000,1460.160000,Jan,2024,573.000000,1348.100000,2024-02-05
4,Canada,Canada,-,Brand,2023-12-29,2024-01-05,631.000000,1485.990000,Jan,2024,575.000000,1371.950000,2024-02-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,Canada,Canada,-,Competitor,2024-12-20,2024-12-27,18.926001,116.965535,Dec,2024,20.918212,143.608506,2024-02-05
1094,Canada,Canada,-,Competitor,2024-12-27,2024-12-28,8.439516,50.085352,Dec,2024,9.346991,61.493218,2024-02-05
1095,Canada,Canada,-,Competitor,2024-12-27,2024-12-29,6.197005,35.450348,Dec,2024,6.823668,43.525254,2024-02-05
1096,Canada,Canada,-,Competitor,2024-12-27,2024-12-30,9.658757,51.022594,Dec,2024,10.582638,62.644178,2024-02-05


In [13]:
LM_SM_three_funnel_df[LM_SM_three_funnel_df['Ds'] >= '2024-01-01'].groupby(['Funnel', 'Month']).sum(numeric_only=1)

OptimalConversionQuantity  OptimalSpendValue  \
Funnel     Month                                                 
Brand      Apr                 19176.000000       43207.200000   
           Aug                 20320.000000       47561.370000   
           Dec                 16899.000000       43064.640000   
           Feb                 14141.000000       33068.510000   
           Jan                 16273.000000       38569.540000   
           Jul                 26861.000000       60071.180000   
           Jun                 22429.000000       50651.050000   
           Mar                 12102.000000       29336.320000   
           May                 12597.000000       30411.650000   
           Nov                 18610.000000       60871.600000   
           Oct                 21002.000000       51861.190000   
           Sep                 22345.000000       52222.610000   
Competitor Apr                  2038.438862        9928.124582   
           Aug                  1767.135820        4150.954640   
           Dec                   976.970586        5030.361503   
           Feb                  2228.294836       13054.817550   
           Jan                  1323.053435        5690.917653   
           Jul                  1803.301006        5014.984422   
           Jun                  1680.506247        5376.166164   
           Mar                  2558.090816       17521.978350   
           May                  1565.875920        3545.687527   
           Nov                   859.301811        5123.016566   
           Oct                  1494.246705        4319.385744   
           Sep                  2009.559111        6513.564958   
Generic    Apr                   699.561138        3690.375418   
           Aug                   590.864180        1191.215360   
           Dec                  2811.029414       15102.288497   
           Feb                   894.705164        5222.082450   
           Jan                  1419.946565        6698.162347   
           Jul                   586.698994        1491.875578   
           Jun                   574.493753        2058.033836   
           Mar                   721.909184        4954.771650   
           May                   617.124080        1532.822473   
           Nov                  2439.698189       14419.533434   
           Oct                   976.753295        3864.514256   
           Sep                   340.440889         968.145042   

                  BaseConversionQuantity     BaseSpend  
Funnel     Month                                        
Brand      Apr              16948.000000  38831.300000  
           Aug              20168.000000  47248.090000  
           Dec              14953.000000  38813.690000  
           Feb              11869.000000  28643.460000  
           Jan              14789.000000  35609.500000  
           Jul              23944.000000  54212.660000  
           Jun              21474.000000  48752.780000  
           Mar              11500.000000  28158.290000  
           May              13283.000000  31767.880000  
           Nov              17554.000000  57883.260000  
           Oct              28933.000000  68966.250000  
           Sep              18834.000000  44966.600000  
Competitor Apr               2097.960978  10818.370679  
           Aug               1790.318248   4467.305408  
           Dec               1062.405534   6176.180386  
           Feb               2317.353110  14441.700035  
           Jan               1409.428825   7050.589354  
           Jul               1798.759986   4901.764554  
           Jun               1683.247766   5426.402090  
           Mar               2660.504866  19049.041080  
           May               1599.467430   4081.851316  
           Nov                840.698913   4866.509394  
           Oct               1607.805864   5878.974972  
           Sep               2091.250483   7646.571859  
Generic    Apr             

In [18]:
LM_SM_three_funnel_df[LM_SM_three_funnel_df['Ds'] >= '2024-02-01'].sum(numeric_only=1)

OptimalConversionQuantity    236717.00
OptimalSpendValue            636402.02
BaseConversionQuantity       230688.00
BaseSpend                    636401.94
dtype: float64

In [15]:
# Construct a BigQuery client object.
bqclient = bigquery.Client()

target_table_name = 'budget_allocation_scenario_with_base_' + region + save_date
table_id = "ups-analytics.forecast_optimization." + target_table_name
print(table_id)
# job_config.
job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")

load_job = bqclient.load_table_from_dataframe(
    LM_SM_three_funnel_df, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = bqclient.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 1098 rows.


table = bqclient.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

In [16]:
LM_SM_three_funnel_df[LM_SM_three_funnel_df.Ds>='2024-01-01'].to_csv(base_path + 'output.csv')